# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#create file path
weather_data = "Weather_Data_Raw.csv"

#import into dataframe
weather_df = pd.read_csv(weather_data)

#display dataframe for reference
weather_df.drop(weather_df.columns[0], axis=1, inplace=True) #gets rid of old index
print(len(weather_df))
weather_df.head()

564


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,innisfail,-17.5333,146.0333,90.212,72,50,2.45,AU,1645489748
1,punta arenas,-53.1500,-70.9167,62.708,51,20,7.20,CL,1645489748
2,qaanaaq,77.4840,-69.3632,-7.024,57,78,4.08,GL,1645489749
3,cape town,-33.9258,18.4232,64.274,76,0,3.09,ZA,1645489728
4,quatre cocos,-20.2078,57.7625,80.096,89,78,7.63,MU,1645489749


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#create lat and long lists for lookups
locations = weather_df[['Lat','Lon']].astype(float)
humidity = weather_df['Humidity'].astype(float)

In [4]:
#Generate heatmap layer for humidity
#create figure object
fig = gmaps.figure()

#create the data layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

#add the data layer to the figure
fig.add_layer(heat_layer)

#display the map
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Weather criteria used: zero cloudiness
cloudless_df = weather_df.loc[weather_df['Cloudiness'] == 0]

#Note: null values not dropped since creation of orignal csv file already removes null rows
print(len(cloudless_df))
cloudless_df.head()

66


,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,cape town,-33.9258,18.4232,64.274,76,0,3.09,ZA,1645489728
8,busselton,-33.6500,115.3333,70.052,61,0,4.16,AU,1645489750
32,thompson,55.7435,-97.8558,-14.638,58,0,3.60,CA,1645489758
48,kapaa,22.0752,-159.3190,79.484,63,0,5.66,US,1645489823
52,bethel,41.3712,-73.4140,42.890,61,0,2.06,US,1645489824


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#create hotel dataframe with hotel name colum
hotel_df = cloudless_df
hotel_df['Hotel Name'] = pd.NaT

#display dataframe
hotel_df.reset_index(inplace=True)
hotel_df

C:\Users\EDUCAT~1\AppData\Local\Temp/ipykernel_12768/2761956482.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = pd.NaT


,index,City,Lat,Lon,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,3,cape town,-33.9258,18.4232,64.274,76,0,3.09,ZA,1645489728,NaT
1,8,busselton,-33.6500,115.3333,70.052,61,0,4.16,AU,1645489750,NaT
2,32,thompson,55.7435,-97.8558,-14.638,58,0,3.60,CA,1645489758,NaT
3,48,kapaa,22.0752,-159.3190,79.484,63,0,5.66,US,1645489823,NaT
4,52,bethel,41.3712,-73.4140,42.890,61,0,2.06,US,1645489824,NaT
...,...,...,...,...,...,...,...,...,...,...,...
61,536,tura,25.5198,90.2201,54.914,63,0,1.74,IN,1645490591,NaT
62,544,havre-saint-pierre,50.2334,-63.5986,10.994,47,0,8.75,CA,1645490593,NaT
63,548,asfi,32.2994,-9.2372,57.344,81,0,6.46,MA,1645490595,NaT
64,553,charters towers,-20.1000,146.2667,86.216,49,0,3.92,AU,1645490656,NaT


In [7]:
#parameters to search for hotels within 5000 meters
hotel_url = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?fields=name%2Cgeometry&input=hotel&inputtype=textquery'

#Set the Hotel Name column to the API values

hotel_ser = []

for Lat,Lon in zip(hotel_df['Lat'],hotel_df['Lon']):
    try:
        query_url = f'{hotel_url}&locationbias=circle%3A5000%40{Lat}%2C{Lon}&key={g_key}'
        response_json = requests.get(query_url).json()
        hotel_ser.append(response_json['candidates'][0]['name'])
    except:
        hotel_ser.append("No Local Hotels Found")

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [17]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

#create the hotel layer as a marker layer with info box content using xml string
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#add the layers to a new image
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))